# Tahu lah apa ini?

In [1]:
# Path to google drive folder
# %cd /content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002. Laporan-eksperimen
# %cd /content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002.Experiment_SWI_16052024/

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest, VotingClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from pyod.models.hbos import HBOS
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, OutlierMixin
from scipy.spatial.distance import mahalanobis

In [3]:
# Load the dataset
# file_path = '01.Dataset-Rofiqul/agriculture_dataset.csv'
# file_path = '01.Dataset-Rofiqul/Ujicoba3-29052024/agriculture_dataset.csv'
# file_path = 'dataset/agriculture_dataset.csv'
# file_path = 'dataset/09_05_1_AgricultureOutliersResult_30052024.csv'
file_path = 'dataset/09_05_3_0_AgricultureOutliersDetectionResults.csv'
data = pd.read_csv(file_path)


In [4]:

# Display the first few rows of the dataset for review
# data.head(),data.info(),data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 2246 non-null   object 
 1   Year                 2246 non-null   int64  
 2   Experiment           2246 non-null   object 
 3   DataUse              2246 non-null   object 
 4   Replication          2246 non-null   object 
 5   Month                2246 non-null   object 
 6   Vegetation           2246 non-null   object 
 7   VegType              2246 non-null   object 
 8   N2O                  2246 non-null   float64
 9   N_rate               2246 non-null   int64  
 10  PP2                  2246 non-null   float64
 11  PP7                  2246 non-null   float64
 12  AirT                 2246 non-null   float64
 13  DAF_TD               2246 non-null   int64  
 14  DAF_SD               2246 non-null   int64  
 15  WFPS25cm             2246 non-null   f

## Detect Nilai NaN

In [5]:

# missing_values_count = data.isna().sum()
# missing_values_count

## Impute NaN or missing value with Mean

In [6]:
# # Impute missing values with the mean of the respective columns
# data['WFPS25cm'].fillna(data['WFPS25cm'].mean(), inplace=True)
# data['NH4'].fillna(data['NH4'].mean(), inplace=True)
# data['NO3'].fillna(data['NO3'].mean(), inplace=True)

# # Verify that there are no missing values left
# missing_values_after_imputation = data.isna().sum()
# missing_values_after_imputation

In [7]:
# Save the updated data with outlier detection results
# data.to_csv('dataset/09_05_2_0_AgricultureNoMissingValueResult.csv', index=False)

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.base import BaseEstimator, ClassifierMixin


In [9]:
# Create a custom wrapper for the individual outlier columns to work with VotingClassifier
class OutlierModelWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, column):
        self.column = column
    
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        return X[self.column]

In [10]:

# Generate synthetic ground truth labels for evaluation purposes
# Creating a synthetic ground truth based on majority voting of outliers
models=['if_outlier', 'lof_outlier','cblof_outlier', 'ocsvm_outlier', 'elliptic_outlier', 'hbos_outlier', 'dbscan_outlier', 'mahalanobis_outlier','zscore_outlier','optics_outlier','iqr_outlier']
data['majority_voting'] = data.apply(lambda row: 1 if row[models].mode()[0] == 1 else -1, axis=1)

# Append the outlier results to the original data
# data = pd.concat([data, data[['IF_outlier', 'LOF_Outlier', 'OCSVM_Outlier', 'Elliptic_Outlier', 'HBOS_Outlier', 'DBSCAN_Outlier', 'mahalanobis_Outlier', 'Majority_Voting']]], axis=1)


In [11]:
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,ocsvm_outlier,elliptic_outlier,hbos_outlier,dbscan_outlier,mahalanobis_outlier,iqr_outlier,zscore_outlier,optics_outlier,cblof_outlier,majority_voting
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,1,1,1,1,1,1,1,1,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,1,1,1,1,1,1,1,1,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,1,1,1,1,1,1,1,-1,1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,1,1,1,1,1,1,1,1,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,1,1,1,1,1,1,1,1,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,1,1,1,1,1,1,1,-1,1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,1,1,1,1,1,1,1,1,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,1,1,1,1,1,1,1,1,1,1


In [12]:

# Save the updated data to the CSV file
data.to_csv('dataset/09_05_4_AgricultureOutliersResult_30052024.csv', index=False)

# Display the updated dataframe with outlier detection results
# print(data.head())
# data

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 2246 non-null   object 
 1   Year                 2246 non-null   int64  
 2   Experiment           2246 non-null   object 
 3   DataUse              2246 non-null   object 
 4   Replication          2246 non-null   object 
 5   Month                2246 non-null   object 
 6   Vegetation           2246 non-null   object 
 7   VegType              2246 non-null   object 
 8   N2O                  2246 non-null   float64
 9   N_rate               2246 non-null   int64  
 10  PP2                  2246 non-null   float64
 11  PP7                  2246 non-null   float64
 12  AirT                 2246 non-null   float64
 13  DAF_TD               2246 non-null   int64  
 14  DAF_SD               2246 non-null   int64  
 15  WFPS25cm             2246 non-null   f

In [14]:
# Creating a synthetic ground truth based on majority voting of outliers
# data['majority'] = data.apply(lambda row: 1 if row[['if_outlier','lof_outlier','ocsvm_outlier','elliptic_outlier','hbos_outlier','dbscan_outlier','mahalanobis_outlier']].mode()[0] == 1 else -1, axis=1)
# Convert outlier labels to binary (1 for outliers, 0 for inliers) for evaluation metrics
y_true = (data['majority_voting'] == -1).astype(int)

In [15]:
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,ocsvm_outlier,elliptic_outlier,hbos_outlier,dbscan_outlier,mahalanobis_outlier,iqr_outlier,zscore_outlier,optics_outlier,cblof_outlier,majority_voting
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,1,1,1,1,1,1,1,1,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,1,1,1,1,1,1,1,1,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,1,1,1,1,1,1,1,-1,1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,1,1,1,1,1,1,1,1,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,1,1,1,1,1,1,1,1,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,1,1,1,1,1,1,1,-1,1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,1,1,1,1,1,1,1,1,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,1,1,1,1,1,1,1,1,1,1


In [16]:


# Voting-based with averaging (soft voting)
# data['average_voting'] = data[[f'{name}_outlier' for name, _ in models]].mean(axis=1).apply(lambda x: -1 if x < 0 else 1)
data['average_voting'] = data[models].mean(axis=1).apply(lambda x: -1 if x < 0 else 1)


In [17]:
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,elliptic_outlier,hbos_outlier,dbscan_outlier,mahalanobis_outlier,iqr_outlier,zscore_outlier,optics_outlier,cblof_outlier,majority_voting,average_voting
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,1,1,1,1,1,1,1,1,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,1,1,1,1,1,1,1,1,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,1,1,1,1,1,1,-1,1,1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,1,1,1,1,1,1,1,1,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,1,1,1,1,1,1,1,1,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,1,1,1,1,1,1,-1,1,1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,1,1,1,1,1,1,1,1,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,1,1,1,1,1,1,1,1,1,1


In [18]:
# Weighted voting
weights = {
    'if_outlier': 1,
    'lof_outlier': 1,
    'ocsvm_outlier': 1,
    'elliptic_outlier': 1,
    'hbos_outlier': 1,
    'dbscan_outlier': 1,
    'mahalanobis_outlier': 1,
    'iqr_outlier':1,
    'zscore_outlier':1,
    'cblof_outlier':1,
    'optics_outlier':1
}
# data['weighted_voting'] = data[[f'{name}_outlier' for name, _ in models]].apply(
# data['weighted_voting'] = data[[models]].apply(
    # lambda row: np.average(row, weights=[weights[name] for name, _ in models]), axis=1).apply(lambda x: -1 if x < 0 else 1)
    # lambda row: np.average(row, weights=[weights[name] for name, _ in models]), axis=1).apply(lambda x: -1 if x < 0 else 1)
data['weighted_voting'] = data[models].apply(
    lambda row: np.average(row, weights=[weights[name] for name in models]), axis=1).apply(lambda x: -1 if x < 0 else 1)

# Calculate the weighted voting process details
# weighted_voting_details = data[models].apply(
    # lambda row: np.average(row, weights=[weights[name] for name in models]), axis=1)

# Add the weighted voting details to the DataFrame
# data['weighted_voting'] = weighted_voting_details.apply(lambda x: -1 if x < 0 else 1)

In [19]:
data['weighted_voting']

0       1
1       1
2       1
3       1
4       1
       ..
2241    1
2242    1
2243    1
2244    1
2245    1
Name: weighted_voting, Length: 2246, dtype: int64

In [20]:
# Save the updated data with outlier detection results
data.to_csv('dataset/09_05_3_2_AgricultureOutliersVotingBasedResult.csv', index=False)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 2246 non-null   object 
 1   Year                 2246 non-null   int64  
 2   Experiment           2246 non-null   object 
 3   DataUse              2246 non-null   object 
 4   Replication          2246 non-null   object 
 5   Month                2246 non-null   object 
 6   Vegetation           2246 non-null   object 
 7   VegType              2246 non-null   object 
 8   N2O                  2246 non-null   float64
 9   N_rate               2246 non-null   int64  
 10  PP2                  2246 non-null   float64
 11  PP7                  2246 non-null   float64
 12  AirT                 2246 non-null   float64
 13  DAF_TD               2246 non-null   int64  
 14  DAF_SD               2246 non-null   int64  
 15  WFPS25cm             2246 non-null   f

In [22]:
# Generate statistics for each detection method and voting-based result
methods = ['if_outlier', 'lof_outlier', 'ocsvm_outlier', 'elliptic_outlier', 'hbos_outlier', 'dbscan_outlier', 'mahalanobis_outlier','cblof_outlier','zscore_outlier','optics_outlier','iqr_outlier', 'average_voting', 'weighted_voting', 'majority_voting']

statistics = {}

for method in methods:
    counts = data[method].value_counts()
    stats = {
        'Inliers': counts.get(1, 0),
        'Outliers': counts.get(-1, 0),
        'Total': counts.sum()
    }
    statistics[method] = stats

# Convert the statistics to a DataFrame for better visualization
statistics_df = pd.DataFrame(statistics).T
statistics_df.to_csv('dataset/09_05_4_0_AgricultureOutliersVotingBasedStatsSummary.csv', index=False)
statistics_df

,Inliers,Outliers,Total
if_outlier,2021,225,2246
lof_outlier,2072,174,2246
ocsvm_outlier,2022,224,2246
elliptic_outlier,2021,225,2246
hbos_outlier,2197,49,2246
dbscan_outlier,2157,89,2246
mahalanobis_outlier,2021,225,2246
cblof_outlier,2021,225,2246
zscore_outlier,2214,32,2246
optics_outlier,1734,512,2246
